# Chess Neural Network

Building a neural network that learns to play chess based on my gameplay data.

## Goal
Train a move prediction model to create an AI that plays like me.




In [1]:
# Necessary libraries
import pandas as pd
import numpy as np
import ast
import torch
import torch.nn as nn
from torch.optim.lr_scheduler import OneCycleLR
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler, SequentialSampler
from sklearn.model_selection import TimeSeriesSplit
import warnings
warnings.filterwarnings('ignore')

## Step 2: Load Chess Data

Load cleaned data from csv file


In [2]:
df = pd.read_csv("/Users/riteshbhandari/Documents/Dokumentit – Ritesh - MacBook Pro/GitHub/Chess-engine/src/data-analysis/cleaned_data.csv")
df.head()

,game_id,moves,num_moves,first_move
0,123118274906,"[('white', 'e4', True), ('black', 'e6', False)...",90,e4
1,123118510404,"[('white', 'd4', False), ('black', 'c5', True)...",141,d4
2,123118790014,"[('white', 'e4', False), ('black', 'e5', True)...",46,e4
3,123158939328,"[('white', 'e4', False), ('black', 'd5', True)...",88,e4
4,123160166430,"[('white', 'e4', True), ('black', 'e5', False)...",110,e4


# Step 2.1:  Further Pre-processing


In [3]:
# convert string representation of list to actual list
df["moves"] = df["moves"].apply(ast.literal_eval)

# saving all the moves to single list to be encoded 
every_move = []
for game in df["moves"]:
    for move in game:
        every_move.append(move[1])

every_move[:10]

['e4', 'e6', 'd4', 'Qh4', 'Nc3', 'f5', 'Nf3', 'Qe7', 'e5', 'Qb4']

In [4]:
# getting all the unique moves
unique_moves = set(every_move)  # just the unique moves
print("Number of different moves:", len(unique_moves))
print()

# give move a number
move_to_number = {}

# turning integer back to moves (for future use)
number_to_move= {}

for i, move in enumerate(unique_moves):
    move_to_number[move] = i
    number_to_move[i] = move

# turning all the numbers into integers
number_moves = []
for move in every_move:
    number_moves.append(move_to_number[move])

# first 10 moves
print("First 10 moves as numbers: ")
print(number_moves[:10])
print()

# first 10 original moves
print("First 10 original moves: ")
print(every_move[:10]) 

Number of different moves: 1927

First 10 moves as numbers: 
[1643, 135, 719, 1647, 248, 404, 1066, 1005, 812, 918]

First 10 original moves: 
['e4', 'e6', 'd4', 'Qh4', 'Nc3', 'f5', 'Nf3', 'Qe7', 'e5', 'Qb4']


In [5]:
# Create colors per game
colors_per_game = []

for game in df["moves"]:
    
    game_colors = []
    for move in game:
        if move[0] == "white":
            game_colors.append(1)
        else:  # black
            game_colors.append(0)
    colors_per_game.append(game_colors)

# Add to DataFrame
df["colors"] = colors_per_game

# Check first row
print(df[["moves", "colors"]].iloc[0])


moves     [(white, e4, True), (black, e6, False), (white...
colors    [1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, ...
Name: 0, dtype: object


In [6]:
# turn white or black to integers ( 1  = (White), 0 = (Black) )
teoriat_moves_per_game = []

for game in df["moves"]:
    
    game_teoriat = []
    for move in game:
        if move[2] == True:
            game_teoriat.append(1)
        else:
            game_teoriat.append(0)
    teoriat_moves_per_game.append(game_teoriat)

# Add to DataFrame
df["teoriat_moves"] = teoriat_moves_per_game

# Check first row
print(df[["moves", "teoriat_moves"]].iloc[0])

moves            [(white, e4, True), (black, e6, False), (white...
teoriat_moves    [1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, ...
Name: 0, dtype: object


In [7]:
#combine all the features into a single list of tuples seperated by games 
combined_features_per_game = []

for game_idx in range(len(df)):
    
    game_combined = []
    moves = df["moves"].iloc[game_idx]
    colors = df["colors"].iloc[game_idx]
    teoriat = df["teoriat_moves"].iloc[game_idx]
    
    for i in range(len(moves)):
        # (color, move_as_integer, your_move)
        game_combined.append((
            colors[i],
            move_to_number[moves[i][1]],  # convert move to integer
            teoriat[i]
        ))
    # Append the combined features for the game
    combined_features_per_game.append(game_combined)

# Add to DataFrame
df["game_data"] = combined_features_per_game

# Check first game data
print(combined_features_per_game[0])

[(1, 1643, 1), (0, 135, 0), (1, 719, 1), (0, 1647, 0), (1, 248, 1), (0, 404, 0), (1, 1066, 1), (0, 1005, 0), (1, 812, 1), (0, 918, 0), (1, 1766, 1), (0, 1057, 0), (1, 1215, 1), (0, 1769, 0), (1, 1176, 1), (0, 20, 0), (1, 1913, 1), (0, 854, 0), (1, 488, 1), (0, 2, 0), (1, 1051, 1), (0, 720, 0), (1, 766, 1), (0, 517, 0), (1, 645, 1), (0, 1873, 0), (1, 1209, 1), (0, 1202, 0), (1, 742, 1), (0, 597, 0), (1, 1051, 1), (0, 1433, 0), (1, 330, 1), (0, 130, 0), (1, 574, 1), (0, 164, 0), (1, 1359, 1), (0, 31, 0), (1, 7, 1), (0, 1745, 0), (1, 455, 1), (0, 752, 0), (1, 831, 1), (0, 612, 0), (1, 574, 1), (0, 1345, 0), (1, 501, 1), (0, 1892, 0), (1, 1908, 1), (0, 1628, 0), (1, 1061, 1), (0, 1268, 0), (1, 1625, 1), (0, 854, 0), (1, 1559, 1), (0, 1086, 0), (1, 574, 1), (0, 539, 0), (1, 7, 1), (0, 1133, 0), (1, 888, 1), (0, 306, 0), (1, 1059, 1), (0, 1221, 0), (1, 272, 1), (0, 356, 0), (1, 203, 1), (0, 1455, 0), (1, 888, 1), (0, 356, 0), (1, 203, 1), (0, 1455, 0), (1, 1214, 1), (0, 611, 0), (1, 1412, 1)

In [8]:
# lets drop the columns with unneccessary data 
df = df.drop(columns=["moves", "colors", "teoriat_moves", "first_move","num_moves"])
df.head()

,game_id,game_data
0,123118274906,"[(1, 1643, 1), (0, 135, 0), (1, 719, 1), (0, 1..."
1,123118510404,"[(1, 719, 0), (0, 1773, 1), (1, 1434, 0), (0, ..."
2,123118790014,"[(1, 1643, 0), (0, 812, 1), (1, 1066, 0), (0, ..."
3,123158939328,"[(1, 1643, 0), (0, 1434, 1), (1, 812, 0), (0, ..."
4,123160166430,"[(1, 1643, 1), (0, 812, 0), (1, 1322, 1), (0, ..."


## Step 2.2: Data Pipeline

### Dataset Class
Organize chess games into training sequences with sliding windows and padding.

In [9]:
unique_moves = set()
for game in df["game_data"]:
    for (color, move, your_move) in game:
        unique_moves.add(move)

print(f"Number of unique moves: {len(unique_moves)}")
print(f"Move range: {min(unique_moves)} to {max(unique_moves)}")

Number of unique moves: 1927
Move range: 0 to 1926


In [10]:
VOCAB_SIZE = 1928 # 1927 moves and one padding token
MAX_SEQUENCE_LENGTH = 6 # for now 
PAD_TOKEN = 1927 # for not known moves

class chessdataset(Dataset):
    
    def __init__(self, games_data, max_seq_len = 6):
        self.max_seq_len = max_seq_len
        self.pad_token = PAD_TOKEN
        self.sequences = []
        self.targets = []
        
        # loop through each games 
        for game in games_data:
            for i in  range(len(game) -1):
                start_idx = max(0,i-5)
                sequence = game[start_idx : i+1]  
            
                while len(sequence) <6:
                    sequence.insert(0,(0,1927,0))
                   
                target = game[i+1][1] # the real seventh move 
                self.sequences.append(sequence)
                self.targets.append(target)



    # training samples
    def __len__ (self):
        return len(self.targets)

        
      # creating batches           
    def __getitem__ (self, idx):
        sequence = self.sequences[idx]
        target = self.targets[idx]

        # saving training samples seperately to it's own category
        colors = [move_tuple[0] for move_tuple in sequence]
        moves = [move_tuple[1] for move_tuple in sequence]
        theory = [move_tuple[2] for move_tuple in sequence]
        
         # total 60 600 training samples
        return {
            'colors': torch.tensor(colors),
            'moves': torch.tensor(moves),
            'theory': torch.tensor(theory),
            'target': torch.tensor(target), 
    }                

In [11]:
# Create the dataset
dataset = chessdataset(df['game_data'])

# Check how many training examples
print(f"Total training examples: {len(dataset)}")

# Get one example
example = dataset[0]
print(f"\nFirst training example:")
print(f"Colors: {example['colors']}")
print(f"Moves: {example['moves']}")
print(f"Theory: {example['theory']}")
print(f"Target: {example['target']}")

print(f"\nShapes:")
print(f"Colors shape: {example['colors'].shape}")
print(f"Moves shape: {example['moves'].shape}")

Total training examples: 60600

First training example:
Colors: tensor([0, 0, 0, 0, 0, 1])
Moves: tensor([1927, 1927, 1927, 1927, 1927, 1643])
Theory: tensor([0, 0, 0, 0, 0, 1])
Target: 135

Shapes:
Colors shape: torch.Size([6])
Moves shape: torch.Size([6])


### Purge K-Fold Cross-Validation
Setup  Purge cross-validation splits for robust model evaluation and for avoiding data leaks. 

In [12]:
# time based split to 5
n_splits = 5

#TimeSeriesSplit (time order)
tscv = TimeSeriesSplit(n_splits=n_splits)

# gives index range for each fold
splits = tscv.split(range(len(dataset)))

# train test split
fold_num = 1

for split in splits:
    train = split[0]
    test = split[1]


    fold_num += 1

## Step 2.3: Build the RNN Model

In [13]:
# Data parameters
BATCH_SIZE = 64
MAX_SEQUENCE_LENGTH = 6
VOCAB_SIZE = 1928
PAD_TOKEN = 1927

# Training parameters
NUM_EPOCHS = 25
LEARNING_RATE = 0.0003  # Changed from 0.001
WEIGHT_DECAY = 0.01

# Model architecture
EMBEDDING_DIM = 128
HIDDEN_DIM = 256
NUM_LAYERS = 2
DROPOUT = 0.3

# Model save path
MODEL_SAVE_PATH = 'best_chess_model.pth'

In [14]:
class chessRNN(nn.Module):
    def __init__(
        self,
        vocab_size=1928,
        embedding_dim=128,
        hidden_dim=256,
        num_layers=2,
        dropout=0.3,
        rnn_type="gru"
    ):
        super(chessRNN, self).__init__()
        
        # Increased embedding size for color and theory from 6 to 32
        self.move_embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=PAD_TOKEN)
        self.color_embedding = nn.Embedding(2, 32)
        self.theory_embedding = nn.Embedding(2, 32)
        
        # Layer normalization for stable training
        input_dim = embedding_dim + 32 + 32
        self.layer_norm = nn.LayerNorm(input_dim)
        
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        
        self.rnn = nn.GRU(
            input_size=input_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            dropout=dropout if num_layers > 1 else 0,
            batch_first=True
        )
        
        # Two-layer output head instead of single layer
        self.dropout = nn.Dropout(dropout)
        self.fc_intermediate = nn.Linear(hidden_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.fc = nn.Linear(hidden_dim, vocab_size)
        
        # Initialize weights properly
        self._init_weights()
    
    def _init_weights(self):
        nn.init.normal_(self.move_embedding.weight, mean=0, std=0.02)
        nn.init.normal_(self.color_embedding.weight, mean=0, std=0.02)
        nn.init.normal_(self.theory_embedding.weight, mean=0, std=0.02)
        nn.init.xavier_uniform_(self.fc_intermediate.weight)
        nn.init.zeros_(self.fc_intermediate.bias)
        nn.init.xavier_uniform_(self.fc.weight)
        nn.init.zeros_(self.fc.bias)
    
    def forward(self, colors, moves, theory):
        move_embedded = self.move_embedding(moves)
        color_embedded = self.color_embedding(colors)
        theory_embedded = self.theory_embedding(theory)
        
        combined_embedded = torch.cat([move_embedded, color_embedded, theory_embedded], dim=2)
        
        # Apply layer normalization
        combined_embedded = self.layer_norm(combined_embedded)
        
        rnn_output, hidden_state = self.rnn(combined_embedded)
        last_hidden = hidden_state[-1, :, :]
        
        # Two-layer output with dropout
        x = self.dropout(last_hidden)
        x = self.fc_intermediate(x)
        x = self.relu(x)
        x = self.dropout(x)
        logits = self.fc(x)
        
        return logits

In [15]:
# Final device setup
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

print(f"Using device: {device}")

# Initialize loss function 
loss_func = nn.CrossEntropyLoss()

Using device: cpu


## 2.4 Training the Model

| **Phase** | **Details** |
|-----------|-------------|
| **Local Testing** | Single fold (Fold 5): 50,500 train / 10,100 validation |
| **Batch Size** | 64 samples per batch |
| **Duration** | 25 epochs  |
| **Target** | 40%+ validation accuracy |
| **Training** | `train_epoch()` - Forward pass → Loss → Backprop → Update weights |
| **Validation** | `validate()` - Evaluate on unseen data (no weight updates) |
| **Checkpointing** | Save best model to `best_chess_model.pth` |
| **Future (CSC)** | 5-fold cross-validation with GPU acceleration |

In [16]:
# Create proper Train/Val/Test split using TimeSeriesSplit
all_folds = list(tscv.split(range(len(dataset))))

# Use Fold 3 for train/val split, Fold 4 for test
train_indices, val_indices = all_folds[2]  # Fold 3: 40,400 train / 10,100 val
_, test_indices = all_folds[3]  # Fold 4: 10,100 test (TRULY UNSEEN)

print(f"Training examples: {len(train_indices)}")
print(f"Validation examples: {len(val_indices)}")  
print(f"Test examples (UNSEEN): {len(test_indices)}")

# Create samplers
train_sampler = SubsetRandomSampler(train_indices)
val_sampler = SequentialSampler(val_indices)
test_sampler = SequentialSampler(test_indices)

# Create DataLoaders
train_loader = DataLoader(dataset, batch_size=BATCH_SIZE, sampler=train_sampler)
val_loader = DataLoader(dataset, batch_size=BATCH_SIZE, sampler=val_sampler)
test_loader = DataLoader(dataset, batch_size=BATCH_SIZE, sampler=test_sampler)

print(f"\nNumber of training batches: {len(train_loader)}")
print(f"Number of validation batches: {len(val_loader)}")
print(f"Number of test batches: {len(test_loader)}")

Training examples: 30300
Validation examples: 10100
Test examples (UNSEEN): 10100

Number of training batches: 474
Number of validation batches: 158
Number of test batches: 158


In [17]:
def train_epoch(model, dataloader, optimizer, loss_func, device, scheduler=None):
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    
    for batch in dataloader: 
        colors = batch['colors'].to(device)
        moves = batch['moves'].to(device)
        theory = batch['theory'].to(device)
        targets = batch['target'].to(device)
        
        optimizer.zero_grad()
        
        logits = model(colors, moves, theory)
        loss = loss_func(logits, targets)
        loss.backward()
        
        # Clip gradients to prevent instability
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        
        optimizer.step()
        
        # Update learning rate if scheduler is provided
        if scheduler is not None:
            scheduler.step()
        
        total_loss += loss.item()
        _, predicted = torch.max(logits, 1)
        correct += (predicted == targets).sum().item()
        total += targets.size(0)
    
    return total_loss / len(dataloader), correct / total

In [18]:
def validate(model, dataloader, loss_func, device):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for batch in dataloader:
            colors = batch['colors'].to(device)
            moves = batch['moves'].to(device)
            theory = batch['theory'].to(device)
            targets = batch['target'].to(device)

            # Logits calculation MUST happen here
            logits = model(colors, moves, theory) 
            
            loss = loss_func(logits, targets)
            
            total_loss += loss.item()
            _, predicted = torch.max(logits, 1)
            correct += (predicted == targets).sum().item()
            total += targets.size(0)

            
    
    return total_loss / len(dataloader), correct / total

In [ ]:
best_val_acc = 0
torch.manual_seed(42)

model = chessRNN(
    vocab_size=VOCAB_SIZE,
    embedding_dim=EMBEDDING_DIM,
    hidden_dim=HIDDEN_DIM,
    num_layers=NUM_LAYERS,
    dropout=DROPOUT
).to(device)

optimizer = optim.AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)

scheduler = OneCycleLR(
    optimizer, 
    max_lr=0.001,
    epochs=NUM_EPOCHS,
    steps_per_epoch=len(train_loader),
    pct_start=0.1
)

print("Starting training...")
print(f"Training on {len(train_indices)} examples")
print(f"Validating on {len(val_indices)} examples")
print(f"Device: {device}\n")

for epoch in range(NUM_EPOCHS):
    train_loss, train_acc = train_epoch(model, train_loader, optimizer, loss_func, device, scheduler)
    val_loss, val_acc = validate(model, val_loader, loss_func, device)
    
    print(f"Epoch {epoch+1}/{NUM_EPOCHS} | Train Loss: {train_loss:.4f}, Acc: {train_acc:.3f} | Val Loss: {val_loss:.4f}, Acc: {val_acc:.3f}")
    
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), MODEL_SAVE_PATH)
        print(f"  ✓ Saved new best: {val_acc:.3f}")

print(f"\n{'='*50}")
print(f"Training complete! Best validation accuracy: {best_val_acc:.3f}")
print(f"Model saved to: {MODEL_SAVE_PATH}")
print(f"{'='*50}")

Starting training...
Training on 30300 examples
Validating on 10100 examples
Device: cpu

Epoch 1/25 | Train Loss: 6.7366, Acc: 0.015 | Val Loss: 6.0860, Acc: 0.027
  ✓ Saved new best: 0.027
Epoch 2/25 | Train Loss: 5.9325, Acc: 0.050 | Val Loss: 5.4689, Acc: 0.069
  ✓ Saved new best: 0.069
Epoch 3/25 | Train Loss: 5.5099, Acc: 0.076 | Val Loss: 4.9924, Acc: 0.116
  ✓ Saved new best: 0.116
Epoch 4/25 | Train Loss: 5.1724, Acc: 0.103 | Val Loss: 4.6603, Acc: 0.146
  ✓ Saved new best: 0.146
Epoch 5/25 | Train Loss: 4.8723, Acc: 0.120 | Val Loss: 4.3156, Acc: 0.172
  ✓ Saved new best: 0.172
Epoch 6/25 | Train Loss: 4.6228, Acc: 0.136 | Val Loss: 4.0170, Acc: 0.210
  ✓ Saved new best: 0.210
Epoch 7/25 | Train Loss: 4.3911, Acc: 0.156 | Val Loss: 3.7129, Acc: 0.242
  ✓ Saved new best: 0.242
Epoch 8/25 | Train Loss: 4.1695, Acc: 0.174 | Val Loss: 3.4216, Acc: 0.286
  ✓ Saved new best: 0.286
Epoch 9/25 | Train Loss: 3.9486, Acc: 0.196 | Val Loss: 3.1407, Acc: 0.332
  ✓ Saved new best: 0.332
E

## 2.5 Testing the model real accuracy

In [ ]:
model.load_state_dict(torch.load(MODEL_SAVE_PATH))
model.eval()

test_loss, test_acc = validate(model, test_loader, loss_func, device)

top_5_correct = 0
total = 0

with torch.no_grad():
    for batch in test_loader:
        logits = model(batch['colors'].to(device), batch['moves'].to(device), batch['theory'].to(device))
        _, top_5 = torch.topk(logits, 5, dim=1)
        
        for i, target in enumerate(batch['target'].to(device)):
            if target in top_5[i]:
                top_5_correct += 1
            total += 1

print(f"\n{'='*50}")
print(f"TEST RESULTS")
print(f"{'='*50}")
print(f"Top-1 Accuracy: {test_acc:.3f} ({test_acc*100:.1f}%)")
print(f"Top-5 Accuracy: {top_5_correct/total:.3f} ({top_5_correct/total*100:.1f}%)")
print(f"{'='*50}")